# Projekt: Lokalizacja punktu w przestrzeni dwuwymiarowej – Metoda Separatorów
**Autor:** Gabriel
**Przedmiot:** Algorytmy Geometryczne

## 1. Wstęp Teoretyczny
[cite_start]Problem lokalizacji punktu polega na znalezieniu regionu $R_i$ w podziale płaszczyzny, który zawiera dany punkt zapytania $p$[cite: 13]. W tym projekcie prezentujemy **Metodę Łańcuchów (Chain Method)**, znaną również jako metoda separatorów.

### Główne założenia metody:
1.  **Monotoniczność:** Podział płaszczyzny składa się z obszarów monotonicznych. [cite_start]Jeśli obszary nie są monotoniczne, wymagana jest regularyzacja (np. triangulacja)[cite: 104].
2.  [cite_start]**Separatory (Łańcuchy):** Konstruujemy zbiór łańcuchów monotonicznych, które uporządkowują obszary od lewej do prawej[cite: 14].

[cite_start]Struktura danych to drzewo binarne, gdzie węzły reprezentują łańcuchy, a liście reprezentują regiony[cite: 30]. Zapytanie o punkt polega na dyskryminacji (porównaniu) położenia punktu względem kolejnych łańcuchów w drzewie, co pozwala na osiągnięcie logarytmicznego czasu zapytania.


In [45]:
import math
import random
from typing import List , Tuple , Optional , Union , Any
from functools import cmp_to_key
from data import raw
import matplotlib as plt
import numpy as np
import pandas as pd
from bitalg.visualizer.main import Visualizer
from time import time
import os

In [46]:
TOLERANCE = 1e-24

# Data Structures

class Vertex:
    def __init__(self, x_coord: float, y_coord: float):
        self.x = x_coord
        self.y = y_coord
        self.adj_out: List[Tuple['Vertex', int]] = []
        self.adj_in: List[Tuple['Vertex', int]] = []
        self.accumulated_weight_in = 0
        self.accumulated_weight_out = 0

    @property
    def coords(self) -> Tuple[float, float]:
        return self.x, self.y

    def __repr__(self):
        return f"V({self.x:.2f}, {self.y:.2f})"

class MonotoneChain:
    def __init__(self):
        self.path_vertices: List[Tuple[float, float]] = []
        self.path_segments: List[Tuple[Tuple[float, float], Tuple[float, float]]] = []

    def add_node(self, coords: Tuple[float, float]):
        self.path_vertices.append(coords)

    def add_segment(self, start: Tuple[float, float], end: Tuple[float, float]):
        self.path_segments.append((start, end))

class SearchTreeNode:
    def __init__(self, segments: List, chain_ref: MonotoneChain, parent: Optional['SearchTreeNode'] = None):
        self.left_child: Optional[SearchTreeNode] = None
        self.right_child: Optional[SearchTreeNode] = None
        self.parent = parent
        self.segments = segments
        self.chain_ref = chain_ref

## 2. Przetwarzanie Grafu i Wagi Planarne
Aby zbudować drzewo łańcuchów, musimy najpierw odpowiednio skierować krawędzie i nadać im wagi. Algorytm wykonuje dwa przejścia:
1.  **Forward pass:** Propagacja wag z dołu do góry.
2.  **Backward pass:** Korekta wag z góry na dół.

[cite_start]Celem jest ustalenie przepływu tak, aby każdy łańcuch mógł zostać wyodrębniony jako ścieżka monotoniczna od źródła do ujścia grafu[cite: 33].

In [47]:
# Geometry Utils

def cross_product(o: Tuple[float, float], a: Tuple[float, float], b: Tuple[float, float]) -> float:
    return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])

# Graph Processing

def build_graph(raw_vertices: List[Tuple[float, float]], raw_edges: List[Tuple[int, int]]) -> List[Vertex]:
    graph_nodes = [Vertex(x, y) for x, y in raw_vertices]
    for start_idx, end_idx in raw_edges:
        # Ensure edges point upwards (or rightwards for equal Y) to maintain DAG property
        u, v = sorted((start_idx, end_idx))
        node_u = graph_nodes[u]
        node_v = graph_nodes[v]
        node_u.adj_out.append((node_v, 1))
        node_v.adj_in.append((node_u, 1))
    return graph_nodes

def compute_planar_weights(graph: List[Vertex]):
    for node in graph:
        center = node.coords
        
        # Sort edges angularly to identify "leftmost" and "rightmost" paths
        def angular_comparator(edge1, edge2):
            p1 = edge1[0].coords
            p2 = edge2[0].coords
            cp = cross_product(center, p1, p2)
            if math.isclose(cp, 0, abs_tol=TOLERANCE): return 0
            return -1 if cp > 0 else 1

        node.adj_out.sort(key=cmp_to_key(angular_comparator))
        node.adj_in.sort(key=cmp_to_key(lambda a, b: -1 * angular_comparator(a, b)))

    # Forward pass: Propagate weights from bottom to top
    for i in range(1, len(graph) - 1):
        v = graph[i]
        v.accumulated_weight_in = sum(w for _, w in v.adj_in)
        v.accumulated_weight_out = len(v.adj_out)
        
        # if input flow > output flow, push excess to leftmost outgoing edge
        if v.accumulated_weight_in > v.accumulated_weight_out:
            target_node, current_w = v.adj_out.pop(0)
            new_weight = current_w + v.accumulated_weight_in - v.accumulated_weight_out
            v.adj_out.insert(0, (target_node, new_weight))
            
            # Update the corresponding incoming edge at the target node
            for idx, (neighbor, w) in enumerate(target_node.adj_in):
                if neighbor == v:
                    target_node.adj_in[idx] = (v, new_weight)
                    break

    # Backward pass: Propagate weights from top to bottom
    for i in range(len(graph) - 2, 0, -1):
        v = graph[i]
        v.accumulated_weight_in = sum(w for _, w in v.adj_in)
        v.accumulated_weight_out = sum(w for _, w in v.adj_out)
        
        # If output flow > input flow, pull excess from leftmost incoming edge
        if v.accumulated_weight_out > v.accumulated_weight_in:
            source_node, current_w = v.adj_in.pop(0)
            new_weight = current_w + v.accumulated_weight_out - v.accumulated_weight_in
            v.adj_in.insert(0, (source_node, new_weight))
            
            for idx, (neighbor, w) in enumerate(source_node.adj_out):
                if neighbor == v:
                    source_node.adj_out[idx] = (v, new_weight)
                    break

## 3. Generowanie Łańcuchów Monotonicznych i Struktura Wyszukiwania
Algorytm zachłannie buduje łańcuchy, "konsumując" wagi krawędzi. Powstałe łańcuchy są następnie organizowane w zbalansowane drzewo poszukiwań binarnego (BST).

* **Liście drzewa:** Odpowiadają regionom.
[cite_start]* **Węzły wewnętrzne:** Odpowiadają łańcuchom (separatorom)[cite: 31].
Drzewo to pozwala na szybkie określenie, po której stronie separatora znajduje się punkt.

In [48]:
def generate_monotone_chains(graph: List[Vertex]) -> List[MonotoneChain]:
    source_node = graph[0]
    total_chains = sum(w for _, w in source_node.adj_out)
    chains = [MonotoneChain() for _ in range(total_chains)]
    
    for chain in chains:
        current_v = source_node
        
        # Traverse graph greedily consuming edge weights
        while current_v.adj_out:
            chain.add_node(current_v.coords)
            chosen_idx = -1
            
            # Pick the rightmost available edge (non-zero weight)
            for i in range(len(current_v.adj_out) - 1, -1, -1):
                neighbor, weight = current_v.adj_out[i]
                if weight > 0:
                    chosen_idx = i
                    break
            if chosen_idx == -1: break
            
            next_v, w = current_v.adj_out[chosen_idx]
            current_v.adj_out[chosen_idx] = (next_v, w - 1) # Decrease capacity
            current_v = next_v
            
        chain.add_node(current_v.coords)
        
        verts = chain.path_vertices
        for k in range(len(verts) - 1):
            chain.add_segment(verts[k], verts[k+1])
    return chains

def create_search_structure(chains: List[MonotoneChain], parent: Optional[SearchTreeNode] = None) -> Optional[SearchTreeNode]:
    if not chains: return None
    mid_idx = len(chains) // 2
    median_chain = chains[mid_idx]
    
    node = SearchTreeNode(median_chain.path_segments, median_chain, parent)
    node.left_child = create_search_structure(chains[:mid_idx], node)
    node.right_child = create_search_structure(chains[mid_idx + 1:], node)
    return node

## 4. Logika Zapytania (Point Location)
Dla zadanego punktu $p$ algorytm schodzi w dół drzewa. W każdym węźle sprawdzamy relację punktu względem separatora:
* Jeśli punkt jest na lewo od separatora -> idziemy do lewego dziecka.
[cite_start]* Jeśli punkt jest na prawo od separatora -> idziemy do prawego dziecka [cite: 49-50].

Test relacji wykorzystuje iloczyn wektorowy (`cross_product`) oraz wyszukiwanie binarne na segmentach separatora.

In [49]:
# Query Logic

def find_position_relative_to_chain(point: Tuple[float, float], node: SearchTreeNode) -> int:
    px, py = point
    segments = node.segments
    target_segment = None
    
    # Binary Search to find the segment covering the point's Y-coordinate
    left_idx, right_idx = 0, len(segments) - 1
    
    while left_idx <= right_idx:
        mid_idx = (left_idx + right_idx) // 2
        p1, p2 = segments[mid_idx]
        
        y_min, y_max = p1[1], p2[1]
        
        if y_min - TOLERANCE <= py <= y_max + TOLERANCE:
            target_segment = (p1, p2)
            break
        elif py < y_min:
            right_idx = mid_idx - 1
        else:
            left_idx = mid_idx + 1
            
    if target_segment is None: 
        return 1
    
    cp = cross_product(target_segment[0], target_segment[1], point)
    
    if math.isclose(cp, 0, abs_tol=TOLERANCE):
        # Handle collinear points: check if strictly inside segment bounds
        min_x = min(target_segment[0][0], target_segment[1][0])
        max_x = max(target_segment[0][0], target_segment[1][0])
        if min_x <= px <= max_x:
            return 0 
        return -1 if px > target_segment[0][0] else 1
    
    # Return -1 for Right, 1 for Left
    return -1 if cp < 0 else 1

def query_search_tree(point: Tuple[float, float], node: Optional[SearchTreeNode], 
                     closest_left: Optional[MonotoneChain] = None,
                     closest_right: Optional[MonotoneChain] = None) -> Union[MonotoneChain, Tuple[MonotoneChain, MonotoneChain]]:
    if node is None:
        return (closest_left, closest_right)
        
    position = find_position_relative_to_chain(point, node)
    
    if position == 0:
        return node.chain_ref
        
    # Traverse BST based on point position relative to current separator
    if position < 0:
        return query_search_tree(point, node.right_child, closest_left=node.chain_ref, closest_right=closest_right)
    else: 
        return query_search_tree(point, node.left_child, closest_left=closest_left, closest_right=node.chain_ref)

def extract_region_edges(chain_a: MonotoneChain, chain_b: MonotoneChain, point: Tuple[float, float]) -> List[Tuple]:
    if chain_a is None: return chain_b.path_segments 
    if chain_b is None: return chain_a.path_segments

    verts_a = chain_a.path_vertices
    verts_b = chain_b.path_vertices
    
    bubbles = []
    i, j = 0, 0
    last_common_a_idx = 0
    last_common_b_idx = 0
    
    # Iterate through both chains to find split and merge points ("bubbles")
    while i < len(verts_a) and j < len(verts_b):
        va = verts_a[i]
        vb = verts_b[j]
        
        is_same = math.isclose(va[0], vb[0], abs_tol=TOLERANCE) and math.isclose(va[1], vb[1], abs_tol=TOLERANCE)
        
        if is_same:
            if i > last_common_a_idx or j > last_common_b_idx:
                # Close the bubble and store it
                bubble_a = verts_a[last_common_a_idx : i+1]
                bubble_b = verts_b[last_common_b_idx : j+1]
                bubbles.append((bubble_a, bubble_b))
                
            last_common_a_idx = i
            last_common_b_idx = j
            i += 1
            j += 1
        else:
            # Advance pointer for the geometrically lower vertex
            if va[1] < vb[1] or (math.isclose(va[1], vb[1]) and va[0] < vb[0]):
                i += 1
            else:
                j += 1
                
    py = point[1]
    result_edges = []
    
    # Find which bubble contains the query point Y-coordinate
    for path_a, path_b in bubbles:
        min_y = min(v[1] for v in path_a + path_b)
        max_y = max(v[1] for v in path_a + path_b)
        
        if min_y <= py <= max_y:
            # Collect unique edges from both sides of the bubble
            for k in range(len(path_a) - 1):
                result_edges.append((path_a[k], path_a[k+1]))
            for k in range(len(path_b) - 1):
                edge = (path_b[k], path_b[k+1])
                if edge not in result_edges:
                    result_edges.append(edge)
            if result_edges:
                return result_edges

    return result_edges

# Main API

def run_point_location(vertices: List[Tuple[float, float]], edges: List[Tuple[int, int]], query_point: Tuple[float, float]):
    graph = build_graph(vertices, edges)
    compute_planar_weights(graph)
    separators = generate_monotone_chains(graph)
    bst_root = create_search_structure(separators)
    
    result = query_search_tree(query_point, bst_root, closest_left=separators[0], closest_right=separators[-1])
    
    # Handle point exactly on a separator
    if isinstance(result, MonotoneChain):
        segments = result.path_segments
        l, r = 0, len(segments) - 1
        while l <= r:
            mid = (l + r) // 2
            seg = segments[mid]
            if seg[0][1] <= query_point[1] <= seg[1][1]:
                return [seg]
            elif query_point[1] < seg[0][1]:
                r = mid - 1
            else:
                l = mid + 1
        return result.path_segments
        
    # Handle point inside a region
    sep_left, sep_right = result
    return extract_region_edges(sep_left, sep_right, query_point)

In [50]:
def separators_method_point_location_algorithm_visualiser(raw_vertices, raw_edges, point):
    try:
        from bitalg.visualizer.main import Visualizer
    except ImportError:
        return None, run_point_location(raw_vertices, raw_edges, point)

    vis = Visualizer()
    vis.add_point(raw_vertices, color="black")
    
    segments = []
    for u, v in raw_edges:
        p1 = raw_vertices[u]
        p2 = raw_vertices[v]
        segments.append((p1, p2))
    vis.add_line_segment(segments, color="gray")
    
    found_edges = run_point_location(raw_vertices, raw_edges, point)
    
    vis.add_point(point, color="green")
    if found_edges:
        vis.add_line_segment(found_edges, color="red", linewidth=3)
        
    return vis, found_edges

def animate_point_location(raw_vertices: List[Tuple[float, float]], 
                           raw_edges: List[Tuple[int, int]], 
                           query_point: Tuple[float, float]):
    """
    Tworzy animację (GIF) działania algorytmu lokalizacji punktu.
    """
    try:
        from bitalg.visualizer.main import Visualizer
    except ImportError:
        print("Brak biblioteki bitalg.")
        return None

    # 1. Budowa struktur
    graph = build_graph(raw_vertices, raw_edges)
    compute_planar_weights(graph)
    separators = generate_monotone_chains(graph)
    bst_root = create_search_structure(separators)
    
    # 2. Inicjalizacja wizualizatora
    vis = Visualizer()
    
    # TŁO: Rysujemy cały graf na szaro (stały element)
    vis.add_point(raw_vertices, color="black", s=5)
    
    all_segments = []
    for u, v in raw_edges:
        p1 = raw_vertices[u]
        p2 = raw_vertices[v]
        all_segments.append((p1, p2))
    vis.add_line_segment(all_segments, color="lightgray", linewidth=1)
    
    # PUNKT: Szukany punkt na zielono (stały element)
    vis.add_point([query_point], color="green", s=20)
    
    # 3. Pętla animacji (przechodzenie przez drzewo)
    current_node = bst_root
    
    while current_node is not None:
        # A. Pokaż cały aktualny SEPARATOR (Pomarańczowy)
        chain_segments = current_node.segments
        # Zapisujemy ID figury, żeby ją potem usunąć
        chain_fig = vis.add_line_segment(chain_segments, color="orange", linewidth=2)
        
        # B. Znajdź konkretny SEGMENT (krawędź) na tym separatorze (Binary Search)
        px, py = query_point
        target_segment = None
        left_idx, right_idx = 0, len(chain_segments) - 1
        
        while left_idx <= right_idx:
            mid_idx = (left_idx + right_idx) // 2
            p1, p2 = chain_segments[mid_idx]
            y_min, y_max = p1[1], p2[1]
            
            if y_min - TOLERANCE <= py <= y_max + TOLERANCE:
                target_segment = (p1, p2)
                break
            elif py < y_min:
                right_idx = mid_idx - 1
            else:
                left_idx = mid_idx + 1
        
        seg_fig = None
        position = 1 # Domyślnie prawo (fallback)

        if target_segment:
            # C. Pokaż sprawdzany SEGMENT (Niebieski, gruby)
            seg_fig = vis.add_line_segment([target_segment], color="blue", linewidth=4)
            
            # Oblicz stronę (logika algorytmu)
            cp = cross_product(target_segment[0], target_segment[1], query_point)
            if math.isclose(cp, 0, abs_tol=TOLERANCE):
                position = 0 # Na krawędzi
            else:
                position = -1 if cp < 0 else 1
        
        # D. USUWANIE (To tworzy "klatkę" animacji)
        if seg_fig:
            vis.remove_figure(seg_fig)
        
        vis.remove_figure(chain_fig)
        
        # E. Decyzja - gdzie idziemy dalej?
        if position == 0:
            break # Znaleziono (na krawędzi)
        elif position < 0:
            current_node = current_node.right_child
        else:
            current_node = current_node.left_child

    # 4. Finał: Pokaż wynikowy obszar na czerwono (zostaje na stałe)
    found_edges = run_point_location(raw_vertices, raw_edges, query_point)
    if found_edges:
        vis.add_line_segment(found_edges, color="red", linewidth=3)
        
    return vis

## 5. Obsługa Dowolnych Wielokątów (Pre-processing)
Ponieważ metoda łańcuchów wymaga regionów monotonicznych, dowolne wielokąty (w tym wklęsłe) muszą zostać poddane obróbce. Stosujemy tutaj metodę **Ear Clipping** do triangulacji wielokątów.

[cite_start]Powstały graf trójkątów może być niespójny (zawierać "wyspy"), dlatego funkcja `patch_graph_connectivity` dodaje wirtualne krawędzie łączące, tworząc spójny graf wymagany przez algorytm[cite: 104].

In [51]:
# ---------------------------------------------------------
# 1. NARZĘDZIA GEOMETRYCZNE (Czysty Python)
# ---------------------------------------------------------
def cross_product(o, a, b):
    """Iloczyn wektorowy 2D (OA x OB)."""
    return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])

def is_point_in_triangle(p, a, b, c):
    """Sprawdza czy punkt p leży wewnątrz trójkąta abc."""
    cp1 = cross_product(a, b, p)
    cp2 = cross_product(b, c, p)
    cp3 = cross_product(c, a, p)
    # Punkt jest w środku, jeśli wszystkie iloczyny mają ten sam znak
    has_neg = (cp1 < 0) or (cp2 < 0) or (cp3 < 0)
    has_pos = (cp1 > 0) or (cp2 > 0) or (cp3 > 0)
    return not (has_neg and has_pos)

def is_convex(prev, curr, next_pt):
    """Sprawdza czy wierzchołek curr jest wypukły (zakładając kolejność CCW)."""
    return cross_product(prev, curr, next_pt) >= 0 # >= 0 dla CCW

# ---------------------------------------------------------
# 2. ALGORYTM EAR CLIPPING (Triangulacja)
# ---------------------------------------------------------
def triangulate_single_polygon(poly_vertices):
    """
    Dzieli pojedynczy wielokąt na trójkąty metodą Ear Clipping.
    Zwraca listę trójek indeksów (lokalnych względem poly_vertices).
    """
    n = len(poly_vertices)
    if n < 3: return []
    
    # Tworzymy listę indeksów, którą będziemy "przycinać"
    indices = list(range(n))
    triangles = []
    
    # Zabezpieczenie przed pętlą nieskończoną
    max_iter = 2 * n * n
    count = 0
    
    while len(indices) > 3:
        if count > max_iter:
            print("Błąd triangulacji: zbyt wiele iteracji (zła topologia?)")
            break
        
        ear_found = False
        num_current = len(indices)
        
        for i in range(num_current):
            # Pobieramy 3 kolejne indeksy (cyklicznie)
            prev_idx = indices[(i - 1) % num_current]
            curr_idx = indices[i]
            next_idx = indices[(i + 1) % num_current]
            
            p_prev = poly_vertices[prev_idx]
            p_curr = poly_vertices[curr_idx]
            p_next = poly_vertices[next_idx]
            
            # 1. Sprawdź czy kąt jest wypukły (to potencjalne ucho)
            if cross_product(p_prev, p_curr, p_next) > 0: # > 0 dla CCW
                
                # 2. Sprawdź czy żaden INNY wierzchołek nie leży w tym trójkącie
                is_ear = True
                for k in range(num_current):
                    test_idx = indices[k]
                    if test_idx in (prev_idx, curr_idx, next_idx):
                        continue
                    
                    if is_point_in_triangle(poly_vertices[test_idx], p_prev, p_curr, p_next):
                        is_ear = False
                        break
                
                # 3. Jeśli to ucho, odetnij je
                if is_ear:
                    triangles.append((prev_idx, curr_idx, next_idx))
                    indices.pop(i)
                    ear_found = True
                    break
        
        count += 1
        if not ear_found:
            # Jeśli nie znaleziono ucha, wielokąt może być zdegenerowany lub CW
            indices.pop(0) 

    # Dodaj ostatni trójkąt
    if len(indices) == 3:
        triangles.append((indices[0], indices[1], indices[2]))
        
    return triangles

# ---------------------------------------------------------
# 3. PATCHING (Naprawa Grafu dla Metody Separatorów)
# ---------------------------------------------------------
def patch_graph_connectivity(vertices, edges):
    """
    Dodaje wirtualne pionowe krawędzie, aby połączyć rozłączne wyspy
    w jeden spójny graf (DAG) od min-y do max-y.
    Rozwiązuje problem 'pop from empty list'.
    """
    # Budujemy listę sąsiedztwa
    adj_out = {i: [] for i in range(len(vertices))}
    adj_in = {i: [] for i in range(len(vertices))}
    
    for u, v in edges:
        p1, p2 = vertices[u], vertices[v]
        if p1[1] < p2[1] or (p1[1] == p2[1] and p1[0] < p2[0]):
            low, high = u, v
        else:
            low, high = v, u
            
        adj_out[low].append(high)
        adj_in[high].append(low)

    sorted_nodes = []
    for i, (x, y) in enumerate(vertices):
        sorted_nodes.append((i, y, x))
    sorted_nodes.sort(key=lambda k: (k[1], k[2]))

    new_edges = list(edges)
    
    # 1. Łączymy Lokalne Źródła (punkty bez wejścia)
    for k in range(1, len(sorted_nodes)): 
        idx, y, x = sorted_nodes[k]
        
        if len(adj_in[idx]) == 0:
            best_candidate = -1
            min_dist = float('inf')
            
            for j in range(k-1, -1, -1):
                cand_idx, cy, cx = sorted_nodes[j]
                dist = abs(cx - x) + (y - cy)*2 
                if dist < min_dist:
                    min_dist = dist
                    best_candidate = cand_idx
                if (y - cy) > 1000: break 

            if best_candidate != -1:
                new_edges.append((best_candidate, idx))
                adj_in[idx].append(best_candidate)

    # 2. Łączymy Lokalne Ujścia (punkty bez wyjścia)
    for k in range(len(sorted_nodes) - 2, -1, -1):
        idx, y, x = sorted_nodes[k]
        
        if len(adj_out[idx]) == 0:
            best_candidate = -1
            min_dist = float('inf')
            
            for j in range(k+1, len(sorted_nodes)):
                cand_idx, cy, cx = sorted_nodes[j]
                dist = abs(cx - x) + (cy - y)*2
                if dist < min_dist:
                    min_dist = dist
                    best_candidate = cand_idx
                if (cy - y) > 1000: break

            if best_candidate != -1:
                new_edges.append((idx, best_candidate))
                adj_out[idx].append(best_candidate)

    return new_edges

# ---------------------------------------------------------
# 4. GŁÓWNA FUNKCJA STERUJĄCA
# ---------------------------------------------------------
def process_polygons_manually(polygons_list):
    global_vertices = []
    triangles_indices = []
    triangle_map = []
    current_offset = 0
    
    # KROK A: Triangulacja każdego poligonu osobno
    for p_id, poly in enumerate(polygons_list):
        for pt in poly:
            global_vertices.append(pt)
        local_tris = triangulate_single_polygon(poly)
        for (a, b, c) in local_tris:
            g_a = a + current_offset
            g_b = b + current_offset
            g_c = c + current_offset
            triangles_indices.append((g_a, g_b, g_c))
            sorted_tri = tuple(sorted((g_a, g_b, g_c)))
            triangle_map.append({'vertices': sorted_tri, 'poly_id': p_id})
        current_offset += len(poly)

    # KROK B: Konwersja trójkątów na unikalne krawędzie
    edges_set = set()
    for (a, b, c) in triangles_indices:
        edges_set.add(tuple(sorted((a, b))))
        edges_set.add(tuple(sorted((b, c))))
        edges_set.add(tuple(sorted((c, a))))
    raw_edges = list(edges_set)
    
    # KROK C: NAPRAWA GRAFU (Patching)
    final_edges = patch_graph_connectivity(global_vertices, raw_edges)
    return global_vertices, final_edges, triangle_map

In [52]:
def manual_delaunay_generator(width=10, height=10, num_points=15):
    """
    Generuje losowe punkty i łączy je w siatkę (Delaunay).
    Zwraca: (vertices, edges, skew)
    """
    # 1. Generowanie punktów
    vertices = []
    # Dodaj narożniki (kluczowe dla stabilności triangulacji na brzegach)
    vertices.extend([(0.0, 0.0), (float(width), 0.0), (0.0, float(height)), (float(width), float(height))])
    
    # Dodaj losowe punkty wewnątrz
    for _ in range(num_points):
        x = random.uniform(0.1, width - 0.1)
        y = random.uniform(0.1, height - 0.1)
        vertices.append((x, y))

    # 2. Triangulacja (Metoda Brute-Force z poprawioną precyzją)
    n = len(vertices)
    edges = set()

    # Sprawdzamy każdą trójkę punktów
    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                x1, y1 = vertices[i]
                x2, y2 = vertices[j]
                x3, y3 = vertices[k]

                # Wyznacznik (podwójne pole trójkąta)
                D = 2 * (x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
                
                # Jeśli punkty są współliniowe (lub prawie), pomiń
                if abs(D) < 1e-5: continue
                
                # Środek okręgu opisanego
                Ux = ((x1**2 + y1**2) * (y2 - y3) + (x2**2 + y2**2) * (y3 - y1) + (x3**2 + y3**2) * (y1 - y2)) / D
                Uy = ((x1**2 + y1**2) * (x3 - x2) + (x2**2 + y2**2) * (x1 - x3) + (x3**2 + y3**2) * (x2 - x1)) / D
                
                r_sq = (Ux - x1)**2 + (Uy - y1)**2
                center = (Ux, Uy)
                
                # Sprawdź Warunek Delaunaya
                is_valid = True
                for m in range(n):
                    if m == i or m == j or m == k: continue
                    dist_sq = (vertices[m][0] - center[0])**2 + (vertices[m][1] - center[1])**2
                    
                    # Zmniejszyliśmy epsilon dla większej tolerancji
                    if dist_sq < r_sq - 1e-5:
                        is_valid = False
                        break
                
                if is_valid:
                    edges.add(tuple(sorted((i, j))))
                    edges.add(tuple(sorted((j, k))))
                    edges.add(tuple(sorted((k, i))))

    edge_list = list(edges)

    # --- ZABEZPIECZENIE (Fallback) ---
    if len(edge_list) == 0:
        print("  [INFO] Triangulacja zwróciła 0 krawędzi. Używam fallbacku.")
        for i in range(len(vertices) - 1):
            edge_list.append((i, i+1))
        # Zamknij pętlę
        edge_list.append((len(vertices)-1, 0))

    return vertices, edge_list, 0

## 6. Przykłady i Testy


In [53]:
Polygons = [
    [
        [ [(np.float64(94.6774193548387), np.float64(6.3311688311688314)), (np.float64(10.64516129032258), np.float64(9.090909090909093)), (np.float64(32.74193548387096), np.float64(22.4025974025974)), (np.float64(64.03225806451613), np.float64(36.36363636363637)), (np.float64(38.38709677419355), np.float64(64.6103896103896)), (np.float64(72.09677419354838), np.float64(75.97402597402598)), (np.float64(15.806451612903224), np.float64(85.55194805194805)), (np.float64(91.29032258064517), np.float64(87.33766233766235))]],
        [(6, 7), (0, 7), (0, 1), (1, 6), (4, 6), (2, 4), (1, 2), (2, 5), (4, 5), (5, 7), (0, 5), (0, 2), (0, 3), (2, 3), (3, 5)]
    ],
    [

    ]
]

In [54]:
P =  [(np.float64(94.6774193548387), np.float64(6.3311688311688314)), (np.float64(10.64516129032258), np.float64(9.090909090909093)), (np.float64(32.74193548387096), np.float64(22.4025974025974)), (np.float64(64.03225806451613), np.float64(36.36363636363637)), (np.float64(38.38709677419355), np.float64(64.6103896103896)), (np.float64(72.09677419354838), np.float64(75.97402597402598)), (np.float64(15.806451612903224), np.float64(85.55194805194805)), (np.float64(91.29032258064517), np.float64(87.33766233766235))]
E =  [(6, 7), (0, 7), (0, 1), (1, 6), (4, 6), (2, 4), (1, 2), (2, 5), (4, 5), (5, 7), (0, 5), (0, 2), (0, 3), (2, 3), (3, 5)]
POINT = (45, 70)
Pol = [[(np.float64(20.161290322580644), np.float64(82.46753246753246)), (np.float64(86.12903225806451), np.float64(85.55194805194805)), (np.float64(74.19354838709677), np.float64(13.149350649350652)), (np.float64(13.387096774193548), np.float64(16.558441558441558))], [(np.float64(45.64516129032258), np.float64(49.837662337662344)), (np.float64(86.12903225806451), np.float64(85.55194805194805)), (np.float64(74.19354838709677), np.float64(13.149350649350652))], [(np.float64(45.64516129032258), np.float64(49.837662337662344)), (np.float64(29.999999999999996), np.float64(28.24675324675325)), (np.float64(20.161290322580644), np.float64(82.46753246753246))], [(np.float64(20.161290322580644), np.float64(82.46753246753246)), (np.float64(50.483870967741936), np.float64(95.12987012987014)), (np.float64(86.12903225806451), np.float64(85.55194805194805))], [(np.float64(86.12903225806451), np.float64(85.55194805194805)), (np.float64(96.29032258064517), np.float64(90.42207792207793)), (np.float64(74.19354838709677), np.float64(13.149350649350652))], [(np.float64(13.387096774193548), np.float64(16.558441558441558)), (np.float64(21.290322580645164), np.float64(28.409090909090907)), (np.float64(25.322580645161292), np.float64(20.454545454545453))], [(np.float64(25.322580645161292), np.float64(20.454545454545453)), (np.float64(13.387096774193548), np.float64(16.558441558441558)), (np.float64(74.19354838709677), np.float64(13.149350649350652))]]
P, E, _ = process_polygons_manually(Pol)

# Próba wizualizacji dla przetworzonych danych
try:
    vis_, e_ = separators_method_point_location_algorithm_visualiser(P, E, POINT)
    vis_.show()
except Exception as e:
    print(f"Wystąpił błąd podczas wizualizacji: {e}")

Wystąpił błąd podczas wizualizacji: pop from empty list


In [55]:
def generate_tri_grid(width, height):
    vertices = []
    edges = []
    for y in range(height + 1):
        for x in range(width + 1):
            vertices.append((float(x), float(y)))
    def get_idx(x, y):
        return y * (width + 1) + x
    for y in range(height):
        for x in range(width):
            u = get_idx(x, y)
            right = get_idx(x + 1, y)
            top = get_idx(x, y + 1)
            top_right = get_idx(x + 1, y + 1)
            edges.append((u, right))
            edges.append((u, top))
            edges.append((right, top_right))
            edges.append((top, top_right))
            edges.append((u, top_right))
    unique_edges = set()
    for u, v in edges:
        if u < v: unique_edges.add((u, v))
        else: unique_edges.add((v, u))
    return vertices, list(unique_edges), 0

def generate_quad_grid(width, height, skew=0.0):
    vertices = []
    edges = []
    for y in range(height + 1):
        shift = y * skew
        for x in range(width + 1):
            vertices.append((float(x + shift), float(y)))
    def get_idx(x, y):
        return y * (width + 1) + x
    for y in range(height + 1):
        for x in range(width):
            u = get_idx(x, y)
            v = get_idx(x + 1, y)
            edges.append((u, v))
    for x in range(width + 1):
        for y in range(height):
            u = get_idx(x, y)
            v = get_idx(x, y + 1)
            edges.append((u, v))
    unique_edges = set()
    for u, v in edges:
        if u < v: unique_edges.add((u, v))
        else: unique_edges.add((v, u))
    return vertices, list(unique_edges), skew

# Scenariusze Testowe
scenarios = [
    {"gen": lambda: manual_delaunay_generator(width=10, height=10, num_points=30), 
        "dims": (10, 10),
        "desc": "Nieregularny (Własna Triangulacja)"},
    {"gen": lambda: generate_quad_grid(3, 3, skew=0.0), "dims": (3, 3), "desc": "Siatka 3x3 (Kwadraty)"},
    {"gen": lambda: generate_quad_grid(4, 2, skew=0.5), "dims": (4, 2), "desc": "Siatka 4x2 (Równoległoboki)"},
    {"gen": lambda: generate_tri_grid(2, 5), "dims": (2, 5), "desc": "Siatka 2x5 (Trójkąty)"},
]

output_folder = "gif"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print("Rozpoczynam generowanie animacji...")
for s_idx, scen in enumerate(scenarios):
    desc = scen["desc"]
    w, h = scen["dims"]
    print(f"\n--- Przetwarzanie scenariusza {s_idx}: {desc} ---")
    vertices, edges, skew = scen["gen"]()
    points_to_check = []
    for _ in range(2):
        py = random.uniform(0, h)
        shift_at_y = py * skew 
        px = random.uniform(0 + shift_at_y, w + shift_at_y)
        points_to_check.append((px, py))
        
    for p_idx, point in enumerate(points_to_check):
        try:
            vis_anim = animate_point_location(vertices, edges, point)
            if vis_anim:
                filename = f"scenariusz_{s_idx:02d}_punkt_{p_idx}.gif"
                full_path = os.path.join(output_folder, filename)
                vis_anim.save_gif(full_path)
                print(f"    -> Zapisano: {full_path}")
        except Exception as e:
            print(f"    [BŁĄD]: {e}")

Rozpoczynam generowanie animacji...

--- Przetwarzanie scenariusza 0: Nieregularny (Własna Triangulacja) ---
    [BŁĄD]: pop from empty list
    [BŁĄD]: pop from empty list

--- Przetwarzanie scenariusza 1: Siatka 3x3 (Kwadraty) ---
    -> Zapisano: gif\scenariusz_01_punkt_0.gif
    -> Zapisano: gif\scenariusz_01_punkt_1.gif

--- Przetwarzanie scenariusza 2: Siatka 4x2 (Równoległoboki) ---
    -> Zapisano: gif\scenariusz_02_punkt_0.gif
    -> Zapisano: gif\scenariusz_02_punkt_1.gif

--- Przetwarzanie scenariusza 3: Siatka 2x5 (Trójkąty) ---
    -> Zapisano: gif\scenariusz_03_punkt_0.gif
    -> Zapisano: gif\scenariusz_03_punkt_1.gif


## 7. Dodatki (Analiza i Input)


In [56]:
def generate_large_procedural_graph(rows, cols):
    """
    Generuje duży, poprawny graf planarny w formie siatki trójkątów.
    Zwraca (vertices, edges).

    Liczba wierzchołków V = rows * cols
    """
    vertices = []
    edges = []

    # 1. Generowanie wierzchołków (V)
    # Dodajemy mały losowy 'jitter', żeby punkty nie były idealnie w linii (bardziej realistyczne)
    for y in range(rows):
        for x in range(cols):
            jitter_x = random.uniform(-0.2, 0.2)
            jitter_y = random.uniform(-0.2, 0.2)
            vertices.append((float(x) + jitter_x, float(y) + jitter_y))

    # Helper do pobierania indeksu w liście jednowymiarowej
    def get_idx(c, r):
        return r * cols + c

    # 2. Generowanie krawędzi (E) - łączenie z sąsiadami (prawo, góra, góra-prawo)
    for r in range(rows - 1):
        for c in range(cols - 1):
            curr = get_idx(c, r)
            right = get_idx(c + 1, r)
            top = get_idx(c, r + 1)
            top_right = get_idx(c + 1, r + 1) # Przekątna

            # Krawędź pozioma
            edges.append((curr, right))
            # Krawędź pionowa
            edges.append((curr, top))
            # Krawędź ukośna (tworzy trójkąty)
            edges.append((curr, top_right))

            # Domknięcie krawędzi na brzegach siatki (dla estetyki, opcjonalne)
            if c == cols - 2: # Ostatnia kolumna, krawędzie pionowe
                edges.append((right, top_right))

        # Ostatni rząd, krawędzie poziome
        for c in range(cols - 1):
            bottom_last = get_idx(c, rows - 1)
            bottom_right_last = get_idx(c + 1, rows - 1)
            edges.append((bottom_last, bottom_right_last))

    return vertices, edges

In [59]:
# Zakładamy, że funkcje build_graph, compute_planar_weights, generate_monotone_chains,
# create_search_structure, query_search_tree oraz generatory są już zdefiniowane.

grid_sizes = [10, 50, 100, 150, 200, 400] #, 600]
result_list = []
ReportsDir = "Reports/"

# Definiujemy konfiguracje testów: nazwa i odpowiadająca jej funkcja generatora
test_configs = [
    ('TriGrid', generate_tri_grid),
    ('QuadGrid', generate_quad_grid)
]

for type_name, generator_func in test_configs:
    print(f"Rozpoczynam benchmark dla: {type_name}")

    for N in grid_sizes:
        # 1. Generowanie danych (używamy generatora z obecnej konfiguracji)
        # Zakładam, że oba generatory zwracają (vertices, edges, _)
        vertices, edges, _ = generator_func(N, N)
        setNum = len(vertices)

        # --- POMIAR CZASU BUDOWY (PREPROCESSING) ---
        t0 = time()
        try:
            graph = build_graph(vertices, edges)
            compute_planar_weights(graph)
            separators = generate_monotone_chains(graph)
            bst_root = create_search_structure(separators)
            build_time = time() - t0

            # --- POMIAR CZASU ZAPYTANIA (QUERY) ---
            query_count = 100
            test_points = [(random.uniform(0, N), random.uniform(0, N)) for _ in range(query_count)]

            t1 = time()
            for p in test_points:
                # query_search_tree używa bst_root i skrajnych separatorów
                query_search_tree(p, bst_root, closest_left=separators[0], closest_right=separators[-1])
            query_time_total = time() - t1
            avg_query_time = query_time_total / query_count

            # Dodanie do listy wyników
            result_list.append({
                'Typ Grafu': type_name,
                'Liczba V': setNum,
                'Liczba E': len(edges),
                'Czas Budowy [s]': build_time,
                'Czas Zapytania [s]': avg_query_time
            })
            print(f"  Zakończono dla V={setNum}")

        except Exception as e:
            print(f"  Błąd podczas przetwarzania {type_name} dla V={setNum}: {e}")

# Tworzenie DataFrame i zapis
final_dataframe = pd.DataFrame(result_list)
print("\nWyniki końcowe:")
print(final_dataframe)

# Zapis do CSV (nadpisze poprzedni plik lub stworzy nowy)
if not os.path.exists(ReportsDir):
    os.makedirs(ReportsDir)

final_dataframe.to_csv(ReportsDir + 'Benchmark_Separatory_Full.csv', index=False)

Rozpoczynam benchmark dla: TriGrid
  Zakończono dla V=121
  Zakończono dla V=2601
  Zakończono dla V=10201
  Zakończono dla V=22801
  Zakończono dla V=40401
  Zakończono dla V=160801
  Zakończono dla V=361201
Rozpoczynam benchmark dla: QuadGrid
  Zakończono dla V=121
  Zakończono dla V=2601
  Zakończono dla V=10201
  Zakończono dla V=22801
  Zakończono dla V=40401
  Zakończono dla V=160801
  Zakończono dla V=361201

Wyniki końcowe:
   Typ Grafu  Liczba V  Liczba E  Czas Budowy [s]  Czas Zapytania (średni) [s]
0    TriGrid       121       320         0.005260                     0.000007
1    TriGrid      2601      7600         0.030554                     0.000012
2    TriGrid     10201     30200         0.136858                     0.000020
3    TriGrid     22801     67800         0.368018                     0.000027
4    TriGrid     40401    120400         2.526286                     0.000029
5    TriGrid    160801    480800         4.113163                     0.000042
6    TriGri

In [58]:
# Zakładamy, że funkcje build_graph, compute_planar_weights, generate_monotone_chains,
# create_search_structure, query_search_tree oraz generatory są już zdefiniowane.

# Lista rozmiarów siatek (NxN), dla których robimy benchmark
grid_sizes = [10, 50, 100, 150, 200, 400 , 600 ]#, 1000 ]#, 10000]
result_list = []

ReportsDir = "Reports/"

for N in grid_sizes:
    # 1. Generowanie danych (Tri Grid)
    # V = N*N, E ≈ 3*N*N
    vertices, edges, _ = generate_tri_grid(N, N)
    setNum = len(vertices)

    # --- POMIAR CZASU BUDOWY (PREPROCESSING) ---
    t0 = time()
    graph = build_graph(vertices, edges)
    compute_planar_weights(graph)
    separators = generate_monotone_chains(graph)
    bst_root = create_search_structure(separators)
    build_time = time() - t0

    # --- POMIAR CZASU ZAPYTANIA (QUERY) ---
    # Wykonujemy np. 100 zapytań, aby uśrednić wynik dla bardzo małych czasów
    query_count = 100
    test_points = [(random.uniform(0, N), random.uniform(0, N)) for _ in range(query_count)]

    t1 = time()
    for p in test_points:
        query_search_tree(p, bst_root, closest_left=separators[0], closest_right=separators[-1])
    query_time_total = time() - t1
    avg_query_time = query_time_total / query_count

    # Dodanie do listy w Twoim stylu
    result_list.append({
        'Liczba V': setNum,
        'Liczba E': len(edges),
        'Czas Budowy [s]': build_time,
        'Czas Zapytania [s]': avg_query_time,
        'Typ Grafu': 'TriGrid'
    })

    # Opcjonalnie powtórz dla QuadGrid, jeśli chcesz mieć oba w jednym pliku
    # vertices_q, edges_q, _ = generate_quad_grid(N, N)
    # ... analogiczny pomiar ...

# Tworzenie DataFrame i zapis
final_dataframe = pd.DataFrame(result_list)
print(final_dataframe)

# Zapis do CSV
final_dataframe.to_csv(ReportsDir + 'Benchmark_Separatory.csv', index=False)

   Liczba V  Liczba E  Czas Budowy [s]  Czas Zapytania (średni) [s] Typ Grafu
0       121       320         0.012304                     0.000005   TriGrid
1      2601      7600         0.028646                     0.000011   TriGrid
2     10201     30200         0.143160                     0.000019   TriGrid
3     22801     67800         0.369196                     0.000023   TriGrid
4     40401    120400         0.600867                     0.000029   TriGrid
5    160801    480800         7.377424                     0.000037   TriGrid
6    361201   1081200        15.703608                     0.000043   TriGrid
